Machine will learn to understand and analyze different ways of the prescriptions written by the doctors.

In [ ]:
# ! pip install scikit-learn==0.23.2

In [2]:
!pip install sklearn-crfsuite

In [31]:
import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from sklearn.metrics import make_scorer,confusion_matrix
from pprint import pprint
from sklearn.metrics import f1_score,classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [1]:
# nltk.download('all')

In [5]:
from sklearn_crfsuite import CRF, metrics
import pycrfsuite

In [6]:
# Input sentences written as the prescriptions
sigs = ["for 5 to 6 days", "inject 2 units", "x 2 weeks", "x 3 days", "every day", "every 2 weeks", "every 3 days", "every 1 to 2 months", "every 2 to 6 weeks", "every 4 to 6 days", "take two to four tabs", "take 2 to 4 tabs", "take 3 tabs orally bid for 10 days at bedtime", "swallow three capsules tid orally", "take 2 capsules po every 6 hours", "take 2 tabs po for 10 days", "take 100 caps by mouth tid for 10 weeks", "take 2 tabs after an hour", "2 tabs every 4-6 hours", "every 4 to 6 hours", "q46h", "q4-6h", "2 hours before breakfast", "before 30 mins at bedtime", "30 mins before bed", "and 100 tabs twice a month", "100 tabs twice a month", "100 tabs once a month", "100 tabs thrice a month", "3 tabs daily for 3 days then 1 tab per day at bed", "30 tabs 10 days tid", "take 30 tabs for 10 days three times a day", "qid q6h", "bid", "qid", "30 tabs before dinner and bedtime", "30 tabs before dinner & bedtime", "take 3 tabs at bedtime", "30 tabs thrice daily for 10 days ", "30 tabs for 10 days three times a day", "Take 2 tablets a day", "qid for 10 days", "every day", "take 2 caps at bedtime", "apply 3 drops before bedtime", "take three capsules daily", "swallow 3 pills once a day", "swallow three pills thrice a day", "apply daily", "apply three drops before bedtime", "every 6 hours", "before food", "after food", "for 20 days", "for twenty days", "with meals"]

# Tokenized words for input sentence
input_sigs = [['for', '5', 'to', '6', 'days'], ['inject', '2', 'units'], ['x', '2', 'weeks'], ['x', '3', 'days'], ['every', 'day'], ['every', '2', 'weeks'], ['every', '3', 'days'], ['every', '1', 'to', '2', 'months'], ['every', '2', 'to', '6', 'weeks'], ['every', '4', 'to', '6', 'days'], ['take', 'two', 'to', 'four', 'tabs'], ['take', '2', 'to', '4', 'tabs'], ['take', '3', 'tabs', 'orally', 'bid', 'for', '10', 'days', 'at', 'bedtime'], ['swallow', 'three', 'capsules', 'tid', 'orally'], ['take', '2', 'capsules', 'po', 'every', '6', 'hours'], ['take', '2', 'tabs', 'po', 'for', '10', 'days'], ['take', '100', 'caps', 'by', 'mouth', 'tid', 'for', '10', 'weeks'], ['take', '2', 'tabs', 'after', 'an', 'hour'], ['2', 'tabs', 'every', '4-6', 'hours'], ['every', '4', 'to', '6', 'hours'], ['q46h'], ['q4-6h'], ['2', 'hours', 'before', 'breakfast'], ['before', '30', 'mins', 'at', 'bedtime'], ['30', 'mins', 'before', 'bed'], ['and', '100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'once', 'a', 'month'], ['100', 'tabs', 'thrice', 'a', 'month'], ['3', 'tabs', 'daily', 'for', '3', 'days', 'then', '1', 'tab', 'per', 'day', 'at', 'bed'], ['30', 'tabs', '10', 'days', 'tid'], ['take', '30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['qid', 'q6h'], ['bid'], ['qid'], ['30', 'tabs', 'before', 'dinner', 'and', 'bedtime'], ['30', 'tabs', 'before', 'dinner', '&', 'bedtime'], ['take', '3', 'tabs', 'at', 'bedtime'], ['30', 'tabs', 'thrice', 'daily', 'for', '10', 'days'], ['30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['take', '2', 'tablets', 'a', 'day'], ['qid', 'for', '10', 'days'], ['every', 'day'], ['take', '2', 'caps', 'at', 'bedtime'], ['apply', '3', 'drops', 'before', 'bedtime'], ['take', 'three', 'capsules', 'daily'], ['swallow', '3', 'pills', 'once', 'a', 'day'], ['swallow', 'three', 'pills', 'thrice', 'a', 'day'], ['apply', 'daily'], ['apply', 'three', 'drops', 'before', 'bedtime'], ['every', '6', 'hours'], ['before', 'food'], ['after', 'food'], ['for', '20', 'days'], ['for', 'twenty', 'days'], ['with', 'meals']]

# Label for each word - Is it a duration, method, qty ,etc.
output_labels = [['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'], ['Method', 'Qty', 'Form'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['EVERY', 'Period'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'TID', 'PO'], ['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'BY', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'AFTER', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Q46H'], ['Q4-6H'], ['Qty', 'PeriodUnit', 'BEFORE', 'WHEN'], ['BEFORE', 'Qty', 'M', 'AT', 'WHEN'], ['Qty', 'M', 'BEFORE', 'WHEN'], ['AND', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'THEN', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN'], ['Qty', 'Form', 'Duration', 'DurationUnit', 'TID'], ['Method', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Qty', 'TIMES', 'Period', 'PeriodUnit'], ['QID', 'Q6H'], ['BID'], ['QID'],['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Duration', 'DurationUnit'], ['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Frequency', 'TIMES', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Period', 'PeriodUnit'], ['QID', 'FOR', 'Duration', 'DurationUnit'], ['EVERY', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['Method', 'Qty', 'Form', 'DAILY'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'DAILY'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['EVERY', 'Period', 'PeriodUnit'], ['BEFORE', 'FOOD'], ['AFTER', 'FOOD'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['WITH', 'FOOD']]

In [7]:
# Small dataset of 56 samples. Actual projects have large dataset.

len(sigs), len(input_sigs) , len(output_labels)

(56, 56, 56)

In [11]:
# This function is to make the tuple with input_words and their labels
# This is not used directly anywhere but just to demonstrate the use of the function
def tuples_maker(input_sigs, output_labels):
    sample_data = []
    for inp, labels in zip(input_sigs, output_labels):
        io_set = []
        for x, y in zip(inp, labels):
            io_set.append((x,y))
        sample_data.append(io_set)
    return sample_data

In [26]:
# Showing the output of the function
tuples_maker(input_sigs, output_labels)

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')],
 [('inject', 'Method'), ('2', 'Qty'), ('units', 'Form')],
 [('x', 'FOR'), ('2', 'Duration'), ('weeks', 'DurationUnit')],
 [('x', 'FOR'), ('3', 'Duration'), ('days', 'DurationUnit')],
 [('every', 'EVERY'), ('day', 'Period')],
 [('every', 'EVERY'), ('2', 'Period'), ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'), ('3', 'Period'), ('days', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('1', 'Period'),
  ('to', 'TO'),
  ('2', 'PeriodMax'),
  ('months', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('2', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('4', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('days', 'PeriodUnit')],
 [('take', 'Method'),
  ('two', 'Qty'),
  ('to', 'TO'),
  ('four', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('2', 'Qty'),
  ('to', 'TO'),
  ('4', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('3', 

In [12]:
# This function is to generate the input_data for training
# We need (input_word, pos, label) format
def triples_maker(sigs, input_sigs, output_labels):
    sample_data = []
    # Taking each sentences and tokenize them.
    for sentence in sigs:
        tokens = word_tokenize(sentence)
        pos = pos_tag(tokens)
        i = sigs.index(sentence)
        sample_data.append((pos,output_labels[i]))
    # Above for loop gives the output as
    # ... [([('for', 'IN'), ('5', 'CD'), ('to', 'TO'), ('6', 'CD'), ('days', 'NNS')],
    # ... ['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit']),
    # Below loop will combine the data.
    combined_samples = []
    for (tuples, labels) in sample_data:
        combined_data = [(inp, pos, label) for ((inp, pos), label) in zip(tuples, labels)]
        combined_samples.append(combined_data)
    return combined_samples

sample_data = triples_maker(sigs, input_sigs, output_labels)
sample_data

# sample_data = triples_maker(whole_data)
# sample_data

[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')],
 [('inject', 'JJ', 'Method'), ('2', 'CD', 'Qty'), ('units', 'NNS', 'Form')],
 [('x', 'RB', 'FOR'),
  ('2', 'CD', 'Duration'),
  ('weeks', 'NNS', 'DurationUnit')],
 [('x', 'RB', 'FOR'),
  ('3', 'CD', 'Duration'),
  ('days', 'NNS', 'DurationUnit')],
 [('every', 'DT', 'EVERY'), ('day', 'NN', 'Period')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('3', 'CD', 'Period'),
  ('days', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('1', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('2', 'CD', 'PeriodMax'),
  ('months', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('4', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('

In [13]:
# Feature extraction

def token_to_features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [14]:
# These functions are also used in predicting the labels

# Get the features (input)
def get_features(doc):
    return [token_to_features(doc,i) for i in range(len(doc))]

# Get the labels (output)
def get_labels(doc):
    return [label for (token, postag, label) in doc]
X = [get_features(doc) for doc in sample_data]
y = [get_labels(doc) for doc in sample_data]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [22]:
%%time
# Submit training data to the trainer

crf = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    crf.append(xseq, yseq)


CPU times: user 2.84 ms, sys: 0 ns, total: 2.84 ms
Wall time: 2.86 ms


In [20]:
# Set the parameters of the model
crf.set_params({
    'max_iterations': 1000,
    'c1': 0.1,
    'c2': 0.01
})

# Providing a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished

crf.train('prescription_parser.model')

In [23]:
# List the parameters
crf.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [24]:
# Making the predictor and opening the model
predictor = pycrfsuite.Tagger()
predictor.open('prescription_parser.model')

In [25]:
# Predicting single input sentence
%%time
y_pred = [predictor.tag(xseq) for xseq in X_test]

CPU times: user 586 µs, sys: 0 ns, total: 586 µs
Wall time: 592 µs


In [82]:
# This function will follow the same process of processing the data.

def predict(sig):
    """
    predict(sig)
    Purpose: Labels the given sig into corresponding labels
    @param sig. A Sentence  # A medical prescription sig written by a doctor
    @return     A list      # A list with predicted labels (first level of labeling)
    >>> predict('2 tabs every 4 hours')
    [['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]
    >>> predict('2 tabs with food')
    [['Qty', 'Form', 'WITH', 'FOOD']]
    >>> predict('2 tabs qid x 30 days')
    [['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]
    """
    X_sample = []
    sample_data = []
    predictions = None
    
    # To find the PoS for each words.
    sample_words = word_tokenize(sig)
    pos_words = pos_tag(sample_words)

    # We need to convert the input data in the format like [input_word, pos, label] because the that's the format used for training
    # We don't have the label so we put the dummy label in the third position
    # When we use get_features function, it will only take input_word and pos and ignore the label
    for words in pos_words:
      single_data = [None, None, None]
      single_data[0] = words[0]
      single_data[1] = words[1]
      single_data[2] = "Label" # Label to be decided
      sample_data.append(single_data)

    # Get the features for sample_data. This will be the input for the model
    X_Sample = [get_features(sample_data)]
    predictor.open('prescription_parser.model')
    predictions = [predictor.tag(xseq) for xseq in X_Sample]
    return predictions

In [86]:
# Sample predictions

input_sentences = ["take 2 tabs every 6 hours x 10 days",
                   "2 capsu for 10 day at bed",
                   "5 days 2 tabs at bed",
                   "3 tabs qid x 10 weeks",
                   "x 30 days",
                   "x 20 months",
                   "take 2 tabs po tid for 10 days",
                   "take 2 capsules po every 6 hours",
                   "inject 2 units pu tid",
                   "swallow 3 caps tid by mouth",
                   "inject 3 units orally",
                   "orally take 3 tabs tid",
                   "by mouth take three caps",
                   "take 3 tabs orally three times a day for 10 days at bedtime",
                   "take 3 tabs orally bid for 10 days at bedtime",
                   "take 3 tabs bid orally at bed",
                   "take 10 capsules by mouth qid",
                   "inject 10 units orally qid x 3 months",
                   "please take 2 tablets per day for a month in the morning and evening each day",
                   "Amoxcicillin QID 30 tablets",
                   "take 3 tabs TID for 90 days with food",
                   "with food take 3 tablets per day for 90 days",
                   "with food take 3 tablets per week for 90 weeks",
                   "take 2-4 tabs",
                   "take 2 to 4 tabs",
                   "take two to four tabs",
                   "take 2-4 tabs for 8 to 9 days",
                   "take 20 tabs every 6 to 8 days",
                   "take 2 tabs every 4 to 6 days",
                   "take 2 tabs every 2 to 10 weeks",
                   "take 2 tabs every 4 to 6 days",
                   "take 2 tabs every 2 to 10 months",
                   "every 60 mins",
                   "every 10 mins",
                   "every two to four months",
                   "take 2 tabs every 3 to 4 days",
                   "every 3 to 4 days take 20 tabs",
                   "once in every 3 days take 3 tabs",
                   "take 3 tabs once in every 3 days",
                   "orally take 20 tabs every 4-6 weeks",
                   "10 tabs x 2 days",
                   "3 capsule x 15 days",
                   "10 tabs"
                   ]

# Printing the results
for input_sentence in input_sentences:
  predictions = predict(input_sentence)
  print(input_sentence)
  print(predictions)
  print("--------------------")

take 2 tabs every 6 hours x 10 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]
--------------------
2 capsu for 10 day at bed
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]
--------------------
5 days 2 tabs at bed
[['Duration', 'DurationUnit', 'Qty', 'Form', 'AT', 'WHEN']]
--------------------
3 tabs qid x 10 weeks
[['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]
--------------------
x 30 days
[['FOR', 'Duration', 'DurationUnit']]
--------------------
x 20 months
[['FOR', 'Duration', 'DurationUnit']]
--------------------
take 2 tabs po tid for 10 days
[['Method', 'Qty', 'Form', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit']]
--------------------
take 2 capsules po every 6 hours
[['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit']]
--------------------
inject 2 units pu tid
[['Method', 'Qty', 'Form', 'Frequency', 'TID']]
--------------------
swallow 3 caps tid by mouth
[['Method', 'Qty', '